In [ ]:
# 1. Subir archivo .sav desde tu computadora al entorno de Google Colab
from google.colab import files # Importa módulo para trabajar con archivos en Colab
uploaded = files.upload() # Abre un cuadro para seleccionar y subir el archivo .sav


In [ ]:
# 2. Leer archivo .sav
!pip install pyreadstat # Instala la librería necesaria para leer archivos SPSS (.sav)
import pyreadstat # Importa la librería para trabajar con archivos .sav
df, meta = pyreadstat.read_sav("BDDenemdu_personas_2024_anual.sav") # # Lee el archivo .sav y separa los datos (df) y metadatos (meta)


In [ ]:
# 2.1 Visualizamos la data
from google.colab import data_table
data_table.DataTable(df)


In [ ]:
# 3. Crear una copia del DataFrame original para no alterar los datos sin etiquetas
df_etiquetado = df.copy()

for var, labels in meta.variable_value_labels.items():
    # Aplica solo si los valores originales son enteros o categorías codificadas
    if var in df.columns and df[var].dtype in ['int64', 'float64', 'object']:
        # Verificamos que los valores actuales son solo códigos presentes en el diccionario
        if df[var].dropna().isin(labels.keys()).all():
            df_etiquetado[var] = df[var].map(labels)

In [ ]:
# 3.1 Visualizamos la data
from google.colab import data_table
data_table.DataTable(df)

In [ ]:
# 4. Crear dataframe con solo las variables necesarias
ecuador = df_etiquetado[['prov', 'pobreza', 'fexp']].copy()


In [ ]:
# 4.1 Visualizamos la data
from google.colab import data_table
data_table.DataTable(ecuador)

In [ ]:
# 5. Renombrar columnas para claridad
ecuador.rename(columns={'prov': 'provincia', 'fexp': 'facpob'}, inplace=True)


In [ ]:
# 6. Asegurar que 'pobreza' y 'provincia' sean tipo categoría
ecuador[['pobreza', 'provincia']] = ecuador[['pobreza', 'provincia']].astype('category')


In [ ]:
# 7. Calcular % de pobreza (FGT0) por provincia
# Agrupa por provincia y categoría de pobreza, y suma los factores poblacionales (facpob) por grupo
tabla = (
    ecuador.groupby(['provincia', 'pobreza'])['facpob'].sum()
    .unstack(fill_value=0) # Convierte las categorías de 'pobreza' en columnas, reemplaza NaN con 0
)

# Calcula el % de personas pobres (fgt0) como la proporción ponderada (usando 'facpob') por provincia
# Redondea el resultado a un decimal
tabla['fgt0'] = (tabla.get('POBRE', 0) / tabla.sum(axis=1) * 100).round(1)


In [ ]:
# 8. Crear tabla final solo con provincia y % de pobreza
tabla_final = tabla[['fgt0']].reset_index()


In [ ]:
# 9. Ver resultado final en Colab
from google.colab import data_table
data_table.DataTable(tabla_final)
